### 라벨 데이터 선별작업

AI-hub에 업로드 된 데이터셋의 크기가 너무 크고   
영상의 중복이 크기 때문에 약 421기가 크기의 화재 영상,라벨 데이터셋을   
약 50기가 크기의 데이터셋으로 줄이기 위함.   
random함수를 통해 약 12~13%의 데이터만 랜덤적으로 선별.   

In [2]:
import zipfile
import numpy as np
import random
import os

parse_dir_list = ['C','S','N']
parse_file_list = ['[라벨]화재씬.zip','[라벨]유사씬.zip','[라벨]무관씬.zip']
whole_label_path = 'C:/Users/aischool/Downloads/화재 발생 예측 영상/Training'
label_dir_path = 'C:/Users/aischool/OneDrive/Desktop/최종프로젝트/dataset/labels'
image_dir_path = 'C:/Users/aischool/OneDrive/Desktop/최종프로젝트/dataset/images'

my_zip_list = []
my_json_list = []
for z_file in parse_file_list:
    try:
        my_zip = zipfile.ZipFile(os.path.join(whole_label_path,z_file))
        my_zip_list.append(my_zip)
        json_list = np.array(my_zip.namelist())
        my_json_list.append(json_list)
        print(f'{z_file}안의 .json 파일의 갯수는 {len(json_list)}개 입니다.')
    except:
        pass

[라벨]화재씬.zip안의 .json 파일의 갯수는 533159개 입니다.


In [40]:
for i in range(len(my_zip_list)):
    if not os.path.exists(label_dir_path + str(i)):
        os.makedirs(label_dir_path + str(i))
    for filename in my_json_list[i]:
        if random.random() < 0.005:
            my_zip_list[i].extract(filename, label_dir_path + str(i))

### json file to txt file for YOLO

json형식으로 저장된 라벨 데이터를 YOLO모델 학습에 적합한 txt파일로 변환하는 과정   
box형식과 polygon형식으로 저장된 라벨 데이터를 변환하여   
전부 box형식의 라벨 데이터로 변환

In [41]:
import os
import zipfile
import numpy as np
import json

for root, dirs, filenames in os.walk('C:/Users/aischool/OneDrive/Desktop/최종프로젝트'):
    for filename in filenames:
        txt = []
        if '.json' in filename:
            data = json.load(open(os.path.join(root,filename),'r',encoding='utf-8'))
            i_w = data['image']['resolution'][0]
            i_h = data['image']['resolution'][1]
            f = open(os.path.join(root,filename.replace('.json','.txt')),'a')
            for v in data['annotations']:
                if 'box' in v:
                    l = np.array(v['box'])
                    try:
                        cx,cy,w,h = ((l[0]+l[2])/2/i_w,(l[1]+l[3])/2/i_h , (l[2]-l[0])/i_w , (l[3]-l[1])/i_h)
                        write_str = str(int(v['class']) - 1)+ ' ' + str(cx.round(6)) + ' ' + str(cy.round(6)) + ' '  + str(w.round(6)) +  ' ' + str(h.round(6)) + '\n'
                        if write_str not in txt:
                            txt.append(write_str)
                    except:
                        print(filename)
                elif 'polygon' in v:
                    l = np.array(v['polygon'])
                    try:
                        maxs = np.max(l,axis=0)
                        mins = np.min(l,axis=0)
                        cx = (maxs[0] + mins[0]) / 2 / i_w
                        cy = (maxs[1] + mins[1]) / 2 / i_h
                        w = (maxs[0] - mins[0]) / i_w
                        h = (maxs[1] - mins[1]) / i_h
                        # print(maxs[0], mins[0], maxs[0], mins[1])
                        # print(cx,cy,w,h)
                        write_str = str(int(v['class']) - 1)+ ' ' + str(cx.round(6)) + ' ' + str(cy.round(6)) + ' '  + str(w.round(6)) +  ' ' + str(h.round(6)) + '\n'
                        if write_str not in txt:
                            txt.append(write_str)
                    except:
                        print(filename)
            for v in txt:
                f.write(v)
            f.close()


### 이미지 데이터 선별작업

앞서 진행했던 라벨 데이터 선별작업을 통해 라벨 데이터는 선별 작업이 끝난 상태   
이제 이미지 데이터에 대한 선별 작업을 진행해야 하는데   
이미지 데이터 선별 작업의 근거는 앞서 진행했던 라벨 데이터를 기준으로 이미지에 대한 라벨의 존재 여부이다.   
이미지에 대한 라벨이 존재하는 경우 압축파일에서 압축 해제를 진행한다.

In [42]:
my_labeled_list = []
for root, dirs, filenames in os.walk('C:/Users/aischool/OneDrive/Desktop/최종프로젝트/dataset'):
    labeled_list = [] #이미지 데이터 선별 작업의 근거가 될 리스트
    for filename in filenames:
        if '.txt' in filename:
            labeled_list.append(filename.replace('.txt','.jpg'))
    if len(labeled_list) > 0:
        my_labeled_list.append(labeled_list)


In [43]:
import zipfile
import numpy as np
for i in range(len(my_labeled_list)):
    for root, dirs, filenames in os.walk('C:/Users/aischool/Downloads/화재 발생 예측 영상/Training'):
        for filename in filenames:
            if '.zip' in filename and '[원천]' in filename:
                print(filename)
                jpg_zip = zipfile.ZipFile(os.path.join(root,filename))
                jpg_list = np.array(jpg_zip.namelist())
                for jpg_name in jpg_list:
                    if jpg_name in my_labeled_list[i]:
                        try:
                            jpg_zip.extract(jpg_name, image_dir_path + str(i)) #'Bad magic number for file header' 오류 발생으로 인한 예외사항 지정
                        except:
                            pass
                    

[원천]화재씬.zip
[원천]화재씬_1.zip
[원천]화재씬_10.zip
[원천]화재씬_11.zip
[원천]화재씬_12.zip
[원천]화재씬_13.zip
[원천]화재씬_2.zip
[원천]화재씬_3.zip
[원천]화재씬_4.zip
[원천]화재씬_5.zip
[원천]화재씬_6.zip
[원천]화재씬_7.zip
[원천]화재씬_8.zip
[원천]화재씬_9.zip


In [44]:
for root, dirs, t_filenames in os.walk('C:/Users/aischool/OneDrive/Desktop/최종프로젝트/dataset'):
    for t_filename in t_filenames:
        if '.txt' in t_filename:
            if not os.path.isfile(os.path.join(root.replace('labels','images'), t_filename.replace('.txt','.jpg'))):
                os.remove(os.path.join(root,t_filename))

### 데이터셋 나누기

이미지-라벨 데이터셋을 train, valid, test = 7:2:1 비율로 분할

In [45]:
import random
import shutil
dir_parse_list1 = ['train','valid','test']
dir_parse_list2 = ['images','labels']

for dpl1 in dir_parse_list1:
    for dpl2 in dir_parse_list2:
        for i in range(len(my_json_list)):
            if not os.path.exists('C:/Users/aischool/OneDrive/Desktop/최종프로젝트/dataset/dataset_' + str(i) + '/' + dpl1 + '/' + dpl2):
                os.makedirs('C:/Users/aischool/OneDrive/Desktop/최종프로젝트/dataset/dataset_' + str(i) + '/' + dpl1 + '/' + dpl2)

for i in range(len(parse_file_list)):
    for root, dirs, filenames in os.walk('C:/Users/aischool/OneDrive/Desktop/최종프로젝트/dataset/images' + str(i)):
        for filename in filenames:
            if random.random() < 0.7:#train : valid : test = 7 : 2 : 1dataset_'+str(i)+'
                shutil.move(os.path.join(root,filename), os.path.join(root[:-1].replace('dataset','dataset/dataset_'+str(i)+'/train'), filename))
                shutil.move(os.path.join(root.replace('images','labels'), filename.replace('.jpg','.txt')), os.path.join(root[:-1].replace('dataset/images','dataset/dataset_'+str(i)+'/train/labels'), filename.replace('.jpg','.txt')))
            else:    
                if random.random() < 0.66: #valid
                    shutil.move(os.path.join(root,filename), os.path.join(root[:-1].replace('dataset','dataset/dataset_'+str(i)+'/valid'), filename))
                    shutil.move(os.path.join(root.replace('images','labels'), filename.replace('.jpg','.txt')), os.path.join(root[:-1].replace('dataset/images','dataset/dataset_'+str(i)+'/valid/labels'), filename.replace('.jpg','.txt')))
                else: #test
                    shutil.move(os.path.join(root,filename), os.path.join(root[:-1].replace('dataset','dataset/dataset_'+str(i)+'/test'), filename))
                    shutil.move(os.path.join(root.replace('images','labels'), filename.replace('.jpg','.txt')), os.path.join(root[:-1].replace('dataset/images','dataset/dataset_'+str(i)+'/test/labels'), filename.replace('.jpg','.txt')))
            

In [ ]:
!python "C:/Users/aischool/OneDrive/Desktop/최종프로젝트/yolov7/train.py" --weights  yolov7-w6.pt --data "data/fire.yaml" --cfg cfg/training/fire.yaml --name fire --hyp data/hyp.scratch.custom.yaml --cache-images